<a href="https://colab.research.google.com/github/BhavyaaaJain/HSCode/blob/main/hscode_with_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
df = pd.read_excel("data.xlsx")

In [139]:
df

,Chapter Number,TariffItem,Description of goods,IGST,Unit,Unnamed: 5
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...",NaN,NaN,NaN
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:",NaN,NaN,NaN
...,...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,0.12,u,NaN


In [140]:
df=df.drop(['IGST', 'Unit', 'Unnamed: 5'], axis=1)
df

,Chapter Number,TariffItem,Description of goods
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...


In [141]:
df.columns=['chapter', 'code', 'desc']
df

,chapter,code,desc
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...


In [142]:
chapter_mapping = {i: chapter for i, chapter in enumerate(df['chapter'].unique())}

In [143]:
chapter_mapping

{0: 'CHAPTER 1',
 1: 'CHAPTER 10',
 2: 'CHAPTER 11',
 3: 'CHAPTER 13',
 4: 'CHAPTER 14',
 5: 'CHAPTER 15',
 6: 'CHAPTER 16',
 7: 'CHAPTER 17',
 8: 'CHAPTER 18',
 9: 'CHAPTER 19',
 10: 'CHAPTER 2',
 11: 'CHAPTER 20',
 12: 'CHAPTER 21',
 13: 'CHAPTER 22',
 14: 'CHAPTER 23',
 15: 'CHAPTER 24',
 16: 'CHAPTER 25',
 17: 'CHAPTER 26',
 18: 'CHAPTER 27',
 19: 'CHAPTER 28',
 20: 'CHAPTER 29',
 21: 'CHAPTER 3',
 22: 'CHAPTER 30',
 23: 'CHAPTER 31',
 24: 'CHAPTER 32',
 25: 'CHAPTER 33',
 26: 'CHAPTER 34',
 27: 'CHAPTER 4',
 28: 'CHAPTER 5',
 29: 'CHAPTER 6',
 30: 'CHAPTER 7',
 31: 'CHAPTER 8',
 32: 'CHAPTER 9',
 33: 'CHAPTER 35',
 34: 'CHAPTER 36',
 35: 'CHAPTER 37',
 36: 'CHAPTER 38',
 37: 'CHAPTER 39',
 38: 'CHAPTER 40',
 39: 'CHAPTER 41',
 40: 'CHAPTER 42',
 41: 'CHAPTER 43',
 42: 'CHAPTER 44',
 43: 'CHAPTER 45',
 44: 'CHAPTER 46',
 45: 'CHAPTER 47',
 46: 'CHAPTER 48',
 47: 'CHAPTER 49',
 48: 'CHAPTER 50',
 49: 'CHAPTER 51',
 50: 'CHAPTER 52',
 51: 'CHAPTER 53',
 52: 'CHAPTER 54',
 53: 'CHAPTE

In [144]:
pattern = r'[^a-zA-Z0-9\s]'
df['desc'] = df['desc'].str.replace(pattern, " ", regex=True)
pattern = r'[\d]'
df['desc'] = df['desc'].str.replace(pattern, '#', regex=True)
df

,chapter,code,desc
0,CHAPTER 1,1012100,Live Horses Asses Mules and Hinnies Horses ...
1,CHAPTER 1,10129,Live Horses Asses Mules and Hinnies Horses ...
2,CHAPTER 1,1012910,Live Horses Asses Mules and Hinnies Horses ...
3,CHAPTER 1,1012990,Live Horses Asses Mules and Hinnies Horses ...
4,CHAPTER 1,10130,Live Horses Asses Mules and Hinnies Asses
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding ### years Of an ...


In [145]:
# stop_words = set(stopwords.words('english'))
# df['desc'] = df['desc'].apply(lambda x: ' '.join([word for word in x if word.lower() not in stop_words]))

In [146]:
df

,chapter,code,desc
0,CHAPTER 1,1012100,Live Horses Asses Mules and Hinnies Horses ...
1,CHAPTER 1,10129,Live Horses Asses Mules and Hinnies Horses ...
2,CHAPTER 1,1012910,Live Horses Asses Mules and Hinnies Horses ...
3,CHAPTER 1,1012990,Live Horses Asses Mules and Hinnies Horses ...
4,CHAPTER 1,10130,Live Horses Asses Mules and Hinnies Asses
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding ### years Of an ...


In [147]:
chapter_keyword_mapping = {}

for chapter_id, chapter in chapter_mapping.items():
    # Filter data for the current chapter
    chapter_data = df[df['chapter'] == chapter]

    # Extract descriptions for the current chapter
    chapter_descriptions = chapter_data['desc']

    # TF-IDF vectorization for the current chapter
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(chapter_descriptions)

    # Get feature names (words)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Calculate TF-IDF scores
    tfidf_scores = tfidf_matrix.sum(axis=0).A1

    # Sort words by TF-IDF score and select the top N as keywords
    num_keywords = 50  # You can adjust this as needed
    top_keywords_indices = tfidf_scores.argsort()[-num_keywords:][::-1]
    top_keywords = [feature_names[i] for i in top_keywords_indices]

    # Create a mapping of keywords to HS codes for the current chapter
    # You would need to replace 'your_hscode' with the actual HS code column
    # keyword_hscode_mapping = {keyword: chapter_data['code'].values[0] for keyword in top_keywords}

    # Store the keyword mapping in the chapter_keyword_mapping dictionary
    chapter_keyword_mapping[chapter_id] = top_keywords

In [148]:

keyword_mapping_for_chapter = chapter_keyword_mapping[3]

In [149]:
keyword_mapping_for_chapter

['resins',
 'and',
 'vegetable',
 'gum',
 'other',
 'natural',
 'gums',
 'oleoresins',
 'extracts',
 'agar',
 'example',
 'balsams',
 'lac',
 'for',
 'saps',
 'or',
 'not',
 'mucilages',
 'derived',
 'whether',
 'modified',
 'from',
 'thickeners',
 'products',
 'of',
 'pectates',
 'pectic',
 'pectinates',
 'substances',
 'seeds',
 'locust',
 'benjamin',
 'tragacanth',
 'guar',
 'bean',
 'beans',
 'arabic',
 'leaves',
 'fruits',
 'spices',
 'flowers',
 'roots',
 'copal',
 'cnsl',
 'storax',
 'myrrh',
 'asian',
 'african',
 'adraganth',
 'cowrie']

In [150]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

In [151]:
def match_to_chapter(user_input, chapters):
    best_match = []
    best_similarity = 0

    # Tokenize the user input
    input_keywords = set(user_input.split())

    for chapter_id, chapter_keywords in chapters.items():
        # Calculate Jaccard similarity between input keywords and chapter keywords
        similarity = jaccard_similarity(input_keywords, chapter_keywords)
        print(chapter_id, similarity)
        # Check if the similarity exceeds the threshold
        if similarity >= best_similarity and similarity!=0:
            best_match.append(chapter_id)
            best_similarity = similarity


    return best_match

In [225]:
matched_chapter = match_to_chapter("wheat", chapter_keyword_mapping)


0 0.0
1 0.02
2 0.02
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 0.0
62 0.0
63 0.0
64 0.0
65 0.0
66 0.0
67 0.0
68 0.0
69 0.0
70 0.0
71 0.0
72 0.0
73 0.0
74 0.0
75 0.0
76 0.0
77 0.0
78 0.0
79 0.0
80 0.0
81 0.0
82 0.0
83 0.0
84 0.0
85 0.0
86 0.0
87 0.0
88 0.0
89 0.0
90 0.0
91 0.0
92 0.0


In [226]:
for i in matched_chapter:
  print(i)
  print(chapter_keyword_mapping[i])

# print(chapter_keyword_mapping[i for i in matched_chapter])

1
['other', 'millet', 'buckwheat', 'cereals', 'canary', 'seeds', 'and', 'seed', 'rice', 'corn', 'wheat', 'meslin', 'or', 'maize', 'rye', 'oats', 'barley', 'milled', 'quality', 'of', 'grain', 'sorghum', 'var', 'mays', 'zea', 'ragi', 'jawar', 'bajra', 'the', 'rough', 'in', 'husk', 'paddy', 'durum', 'not', 'wholly', 'glazed', 'whether', 'polished', 'semi', 'quinoa', 'indenta', 'dent', 'triticale', 'broken', 'husked', 'brown', 'spp', 'digitaria', 'fordo']
2
['of', 'or', 'starches', 'other', 'meal', 'and', 'the', 'heading', 'groats', 'inulin', 'cereal', 'flour', 'wheat', 'pellets', 'sago', 'flaked', 'rolled', 'than', 'that', 'flours', 'meslin', 'chapter', 'products', 'rice', 'roots', 'tubers', 'powder', 'leguminous', 'dried', 'vegetables', 'starch', 'grains', 'cereals', 'roasted', 'corn', 'maize', 'for', 'example', 'hulled', 'kibbled', 'worked', 'sliced', 'pearled', 'not', 'whole', 'ground', 'germ', 'flakes', 'granules', 'otherwise']


In [227]:
!pip install tensorflow_text

In [228]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [229]:
if len(matched_chapter)==0:
    for chapter_id, chapter in chapter_mapping.items():
      matched_chapter.append(chapter_id)

matched_chapter


[1, 2]

In [230]:
matched_embeddings={}
def encode_text_matched():
  for ch in matched_chapter:
    chapter_data = df[df['chapter']==chapter_mapping[ch]]
    for ind, row in chapter_data.iterrows():
      text = row['desc']
      # print(text)
      preprocessed_text = bert_preprocess([text])
      # print(preprocessed_text)
      bert_encoded = bert_encoder(preprocessed_text)['pooled_output']
      # print(bert_encoded)
      matched_embeddings[row['code']] = bert_encoded



In [231]:
encode_text_matched()


In [232]:
def encode_input(input):
  preprocessed_text = bert_preprocess([input])
  bert_encoded = bert_encoder(preprocessed_text)['pooled_output']
  return bert_encoded

In [233]:
input="wheat"

In [234]:
input_encoded=encode_input(input)

In [235]:
input_encoded

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.90395474, -0.26008552,  0.26870057,  0.78655404, -0.01643089,
        -0.16257171,  0.9133777 ,  0.21160552, -0.00499471, -0.9999612 ,
        -0.26263168,  0.5671145 ,  0.98054   , -0.11595776,  0.93034166,
        -0.6851171 , -0.20805012, -0.602268  ,  0.43957967, -0.76774555,
         0.59424406,  0.9885333 ,  0.40606615,  0.3165639 ,  0.41230565,
         0.72823673, -0.6577857 ,  0.92687577,  0.9512758 ,  0.79522437,
        -0.6546148 ,  0.22509633, -0.9863531 , -0.26496965,  0.07829618,
        -0.9877588 ,  0.20129672, -0.8245013 , -0.15527222, -0.02316396,
        -0.91332054,  0.31126192,  0.99946266,  0.4342295 ,  0.1596282 ,
        -0.35878283, -0.99998826,  0.27282563, -0.8922701 ,  0.09667671,
         0.24288689, -0.34761637,  0.15720859,  0.5024995 ,  0.45787418,
         0.14470115, -0.06121332,  0.16053401, -0.16157918, -0.55908966,
        -0.5613361 ,  0.4119561 , -0.21230066, -0.9178997 ,  0.16920465,
 

In [236]:
input_encoded.shape

TensorShape([1, 768])

In [237]:
input_encoded.reshape(1, -1)

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.90395474, -0.26008552,  0.26870057,  0.78655404, -0.01643089,
        -0.16257171,  0.9133777 ,  0.21160552, -0.00499471, -0.9999612 ,
        -0.26263168,  0.5671145 ,  0.98054   , -0.11595776,  0.93034166,
        -0.6851171 , -0.20805012, -0.602268  ,  0.43957967, -0.76774555,
         0.59424406,  0.9885333 ,  0.40606615,  0.3165639 ,  0.41230565,
         0.72823673, -0.6577857 ,  0.92687577,  0.9512758 ,  0.79522437,
        -0.6546148 ,  0.22509633, -0.9863531 , -0.26496965,  0.07829618,
        -0.9877588 ,  0.20129672, -0.8245013 , -0.15527222, -0.02316396,
        -0.91332054,  0.31126192,  0.99946266,  0.4342295 ,  0.1596282 ,
        -0.35878283, -0.99998826,  0.27282563, -0.8922701 ,  0.09667671,
         0.24288689, -0.34761637,  0.15720859,  0.5024995 ,  0.45787418,
         0.14470115, -0.06121332,  0.16053401, -0.16157918, -0.55908966,
        -0.5613361 ,  0.4119561 , -0.21230066, -0.9178997 ,  0.16920465,
 

In [238]:
similarities={}
for code, embeddings in matched_embeddings.items():
  embeddings = embeddings.reshape(1, -1)
  # print(embeddings.shape)
  similarity_score = cosine_similarity(input_encoded, embeddings)[0][0]
  chapter_data = df[df['code']==code]
  for ind, row in chapter_data.iterrows():
    text = row['desc']
    similarities[code]=[text, similarity_score]

In [239]:
sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

In [240]:
top_5_similar_hs_codes = sorted_similarities[:5]

In [241]:
top_5_similar_hs_codes

[('10019100', ['Wheat and meslin  Other  Seed', 0.79659945]),
 ('10019910', ['Wheat and meslin  Other  Other  Wheat', 0.78123283]),
 ('10019920', ['Wheat and meslin  Other  Other  Meslin', 0.76965165]),
 ('100199', ['Wheat and meslin  Other  Other  ', 0.7640067]),
 ('10011100', ['Wheat and meslin  Durum wheat  Seed', 0.7894159])]